<img src="img/csdms_logo.jpg">

# BMI Live!

Visualize the output of the BMI-ed Diffusions model.

## Setup

Need to install:
* numpy
* pandas
* matplotlib
* bokeh?
* xarray
* holoviews

In [ ]:
import os
import numpy as np
import holoviews as hv
hv.extension('bokeh', 'matplotlib')

Start by importing the `BmiDiffusion` class from the **bmi-live-2018** package and creating an instance:

In [ ]:
from bmi_live.bmi_diffusion import BmiDiffusion
x = BmiDiffusion()

Locate a configuration file for this example and use it to initialize the model:

In [ ]:
from bmi_live import data_directory
cfg_file = os.path.join(data_directory, 'visualize_diffusion.yaml')
x.initialize(cfg_file)

In [ ]:
temp = x.get_value('plate_surface__temperature')

Add a unit impulse to the initial temperature field: 

In [ ]:
temp[200] = 100.0
x.set_value('plate_surface__temperature', temp)

Get the id of the grid on which the temperature variable is located:

In [ ]:
grid_id = x.get_var_grid('plate_surface__temperature')
grid_id

Use the grid id to get the shape of the grid:

In [ ]:
grid_shape = x.get_grid_shape(grid_id)
grid_shape

Set the number of time steps that we'll advance the model:

In [ ]:
n_steps = 35

Make a variable to hold the values of the temperature field over time:

In [ ]:
temperature = np.zeros((*grid_shape, n_steps))

What is the shape of this variable?

In [ ]:
temperature.shape

Locate the starting perturbed temperature field into the `temperature` variable:

In [ ]:
temperature[:,:,0] = temp.reshape(grid_shape)

Advance the model through time:

In [ ]:
for t in range(n_steps):
    x.update()
    next_temp = x.get_value('plate_surface__temperature')
    temperature[:,:,t] = next_temp.reshape(grid_shape)

In [ ]:
ds = hv.Dataset((np.arange(n_steps), np.arange(grid_shape[1]), np.arange(grid_shape[0]),
                 temperature), ['Time','x', 'y'], 'Temperature')
ds

In [ ]:
type(ds.data), list(ds.data.keys())

In [ ]:
ds.clone(datatype=['xarray']).data

In [ ]:
%opts Image (cmap='Reds')[colorbar=True]
%output size=150
ds.to(hv.Image, ['x', 'y'], dynamic=True)